#### note : สำหรับคนที่ใช้ macbook silicon chip ให้ลงตาม link นี้ก่อน

https://piyawat-c.notion.site/How-to-install-Transformers-on-Apple-Silicon-Chip-519b29b0217a47bba22ac55f4b92bd5b

## Pre Install

In [1]:
# !pip install llama-index-vector-stores-faiss
# !pip install llama-index
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-huggingface
# !pip install llama-index-llms-llama-cpp
# !pip install llama-index-readers-wikipedia
# !pip install wikipedia
# !pip install faiss-cpu

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    Settings,
    load_index_from_storage,
    ServiceContext,
    SimpleDirectoryReader,
    VectorStoreIndex
)
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

import wikipedia
wikipedia.set_lang('th')

import json
from IPython.display import Markdown, display

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

## Load Model

In [3]:
embedding_model_name = "kornwtp/SCT-KD-model-phayathaibert" #bge-m3
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=512 , device='cpu')

## Load data

In [4]:
reader = WikipediaReader()

In [5]:
cities = [
    "จังหวัดอยุธยา",
    "จังหวัดปทุมธานี",
    "จังหวัดนครราชสีมา",
    "จังหวัดเชียงใหม่",
    "จังหวัดนครศรีธรรมราช",
    ]

In [6]:
documents = reader.load_data(pages=cities)

In [7]:
documents[0].text[:100]

'พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมขอ'

In [8]:
Document(text=documents[0].text)

Document(id_='c08abeac-3ad2-4844-b8ed-9c6ad6564012', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='พระนครศรีอยุธยา เป็นจังหวัดหนึ่งในภาคกลางและเป็นเขตเศรษฐกิจอุตสาหกรรมที่สำคัญ โดยมีผลิตภัณฑ์มวลรวมของจังหวัดมีมูลค่าสูงเป็นอันดับ 3 ของประเทศ และมีประวัติศาสตร์ความเป็นมาที่ยาวนาน เคยมีชื่อเสียงเป็นแหล่งปลูกข้าวที่สำคัญ จังหวัดพระนครศรีอยุธยาเป็นจังหวัดเดียวในประเทศไทยที่ไม่มีอำเภอเมือง แต่มี อำเภอพระนครศรีอยุธยา เป็นศูนย์กลางการบริหารจัดการด้านต่าง ๆ ชาวบ้านโดยทั่วไปนิยมเรียกอีกชื่อว่า "กรุงเก่า" หรือ "เมืองกรุงเก่า" ห่างจากกรุงเทพมหานครประมาณ 75 กิโลเมตร\n\n\n== ประวัติศาสตร์ ==\n\nพระนครศรีอยุธยาเคยเป็นราชธานี (เมืองหลวง) ของอาณาจักรอยุธยา หรืออาณาจักรสยาม ตลอดระยะเวลา 417 ปี ตั้งแต่ พ.ศ. 1893 กระทั่งเสียกรุงแก่พม่า เมื่อ พ.ศ. 2310 ครั้นเมื่อพระเจ้าตากสินมหาราชทรงสถาปนาราชธานีแห่งใหม่ที่กรุงธนบุรี กรุงศรีอยุธยาก็ไม่ได้กลายเป็นเมืองร้าง ยังมีคนที่รักถิ่นฐานบ้านเดิมอาศัยอยู่และมีราษฎรที่หลบหนีไปอยู่ตามป่ากลับเข้ามาอาศ

## Indexing

In [9]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=200, chunk_overlap=10),
        embed_model,
    ]
)

## run the pipeline
# nodes = pipeline.run(documents=[Document(text=data)])
nodes = pipeline.run(documents=documents)

In [10]:
dim = 768
faiss_index = faiss.IndexFlatL2(dim)

In [11]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes=nodes,storage_context=storage_context, service_context = service_context)

/var/folders/y5/9ws4vr090mdg04y2t969l9pm0000gn/T/ipykernel_3920/3539845662.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


LLM is explicitly disabled. Using MockLLM.


In [12]:
persist_dir="./province"

In [13]:
index.storage_context.persist(persist_dir)

In [14]:
vector_store = FaissVectorStore.from_persist_dir(persist_dir)

In [15]:
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context, embed_model=embed_model)

In [16]:
retriever = index.as_retriever(similarity_top_k=5)

# RAG

## {R} : Retrieve

In [17]:
def retrieve_query(question):
    nodes_ = retriever.retrieve(question)
    for i in range(5):
        print(f'Score:{nodes_[i].score}')
        print(f'Context:{nodes_[i].text}')
        print('='*100)

In [18]:
question = 'แนะนำปทุมธานี'

In [19]:
retrieve_query(question)

Score:0.8553544282913208
Context:== ดูเพิ่ม ==
จังหวัดธัญบุรี
รายชื่อโรงเรียนในจังหวัดปทุมธานี
รายชื่อห้างสรรพสินค้าในจังหวัดปทุมธานี


== อ้างอิง ==
Score:0.8700897097587585
Context:=== การศึกษา ===
รายชื่อสถาบันการศึกษาในจังหวัดปทุมธานีมีดังนี้

โรงเรียน
ดูที่ รายชื่อโรงเรียนในจังหวัดปทุมธานี
สถาบันวิจัย
Score:0.9015171527862549
Context:ปทุมธานี (เดิมสะกดว่า ประทุมธานี) เป็นจังหวัดหนึ่งที่อยู่ในภาคกลางของประเทศไทย เป็นหนึ่งในห้าจังหวัดในพื้นที่ปริมณฑลของกรุงเทพมหานคร ตั้งอยู่บริเวณลุ่มแม่น้ำเจ้าพระยาทางทิศเหนือของกรุงเทพมหานคร
Score:0.9075682163238525
Context:อำเภอเมืองปาน อำเภอเมืองลำปาง (จังหวัดลำปาง) อำเภอบ้านธิ อำเภอเมืองลำพูน อำเภอป่าซาง อำเภอเวียงหนองล่อง  อำเภอบ้านโฮ่ง และอำเภอลี้ (จังหวัดลำพูน)
Score:0.9133187532424927
Context:รองลงมาคืออำเภอเมืองปทุมธานี อำเภอลำลูกกา อำเภอธัญบุรี อำเภอลาดหลุมแก้ว และ อำเภอสามโคก ส่วนพื้นที่ที่มีโรงงานน้อยที่สุดได้แก่ อำเภอหนองเสือ
ตลาดขายส่งที่สำคัญในจังหวัดปทุมธานี ได้แก่ ตลาดไท


## {A} : Augment

In [20]:
Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [21]:
vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)

In [22]:
query_enging = index.as_query_engine()

In [23]:
res = query_enging.query(question)

In [24]:
res

Response(response='Context information is below.\n---------------------\n== ดูเพิ่ม ==\nจังหวัดธัญบุรี\nรายชื่อโรงเรียนในจังหวัดปทุมธานี\nรายชื่อห้างสรรพสินค้าในจังหวัดปทุมธานี\n\n\n== อ้างอิง ==\n\n=== การศึกษา ===\nรายชื่อสถาบันการศึกษาในจังหวัดปทุมธานีมีดังนี้\n\nโรงเรียน\nดูที่ รายชื่อโรงเรียนในจังหวัดปทุมธานี\nสถาบันวิจัย\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: แนะนำปทุมธานี\nAnswer: ', source_nodes=[NodeWithScore(node=TextNode(id_='8270d8d1-855e-4d5e-804c-9bbb28171efb', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7136', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='b5dff15c8089952e9e4876ffbed044717600a9324a7f4b828e2bf6358245efb8'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='1ed24a7c-c61a-46a5-b5b4-d784555d423b', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='cfc1cb

In [29]:
print(res.response)

Context information is below.
---------------------
== ดูเพิ่ม ==
จังหวัดธัญบุรี
รายชื่อโรงเรียนในจังหวัดปทุมธานี
รายชื่อห้างสรรพสินค้าในจังหวัดปทุมธานี


== อ้างอิง ==

=== การศึกษา ===
รายชื่อสถาบันการศึกษาในจังหวัดปทุมธานีมีดังนี้

โรงเรียน
ดูที่ รายชื่อโรงเรียนในจังหวัดปทุมธานี
สถาบันวิจัย
---------------------
Given the context information and not prior knowledge, answer the query.
Query: แนะนำปทุมธานี
Answer: 


## {G} : Generation

In [30]:
# url = 'http://10.223.72.15:8353/completion'
url = 'http://127.0.0.1:8080/completion'

In [31]:
import requests
def send_api2(prompt,
             url = url,
             temp = 0.7,
             n_predict= 512,
            ):
    payload = json.dumps({
    "prompt": f'''<s>[INST] <<SYS>>
    You are a question answering assistant. Answer the question as truthful and helpful as possible
    คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด
    <</SYS>>

    Answer the question based only on the following context:
    {prompt}

    [/INST]''',
    "n_predict": n_predict,
    'temperature': temp,
    "stop": ["</s>"]
    })
    headers = {
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    
    return response

In [32]:
def generate_answer(question, url):
    res = query_enging.query(question)
    res = send_api2(prompt=res.response, url=url)
    return res

In [33]:
question = 'สถานที่เที่ยวปทุมธานี'

In [34]:
%%time
r = generate_answer(question, url) # 7b

CPU times: user 924 ms, sys: 928 ms, total: 1.85 s
Wall time: 29.8 s


In [38]:
print(r.json()['content'])

ยินดีค่ะ จากข้อมูลที่ให้มา จุดหมายปลายทางที่ต้องไปเยือนเมื่อไปเที่ยวปทุมธานี อันดับแรกเลยคือ "วัดบางพลีใหญ่" วัดเก่าแก่ที่สุดในจังหวัด มีสถาปัตยกรรมที่งดงามแบบไทยโบราณ ทั้งวิหารหลวง หอพระนารายณ์ทรงธรรม และเจดีย์ต่างๆ 

ต่อมาคือ "วัดสระปทุม" เป็นวัดเก่าแก่อีกแห่งหนึ่ง มีบรรยากาศเงียบสงบ เหมาะแก่การทำสมาธิและนั่งสมาธิ "สวนสัตว์เปิดเขาเขียว" เป็นสวนสัตว์เอกชนที่มีชื่อเสียงของปทุมธานี ที่คุณสามารถเพลิดเพลินไปกับการชมสัตว์นานาชนิดและธรรมชาติที่สวยงาม 

สำหรับผู้ที่ชื่นชอบการถ่ายภาพสถานที่ท่องเที่ยว แนะนำให้ไปที่ "บ้านวิมานเมฆ" บ้านพักตากอากาศที่สวยงามและมีประวัติศาสตร์อันยาวนาน และ "พิพิธภัณฑ์บ้านจิม ทอมป์สัน" จัดแสดงผลงานศิลปะมากมาย ทั้งภาพวาด ประติมากรรม เฟอร์นิเจอร์ และภาพถ่าย 

ในส่วนของแหล่งช้อปปิ้งและร้านอาหารนั้น คุณสามารถเลือกเดินทางไปที่ "ตลาดนัดรถไฟ" ตลาดนัดขนาดใหญ่ที่รวบรวมสินค้ามือสองและของเก่าหายาก และ "ร้านอาหารอรรถไท" ร้านอาหารที่โดดเด่นด้านอาหารไทยต้นตำรับ


In [36]:
res = query_enging.query(question)

In [37]:
res.response

'Context information is below.\n---------------------\nปทุมธานี (เดิมสะกดว่า ประทุมธานี) เป็นจังหวัดหนึ่งที่อยู่ในภาคกลางของประเทศไทย เป็นหนึ่งในห้าจังหวัดในพื้นที่ปริมณฑลของกรุงเทพมหานคร ตั้งอยู่บริเวณลุ่มแม่น้ำเจ้าพระยาทางทิศเหนือของกรุงเทพมหานคร\n\nรวมทั้งการอนุรักษ์ การถ่ายทอดอารยธรรม ให้เป็นที่รู้จักของอำเภอใกล้เคียง และประชาชนชาวไทย ให้ได้รับรู้ถึงความเป็นมาของอำเภอโนนสูง รวมทั้งเป็นแหล่งท่องเที่ยวทั้งการจัดการท่องเที่ยวแบบโฮมสเตย์ นอกจากนี้\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: สถานที่เที่ยวปทุมธานี\nAnswer: '